In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/C247Project/project/

/content/drive/.shortcut-targets-by-id/1lO_lCKfnzu9fB3MDutIRFUOktcz9B9el/C247Project/project


In [3]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 2.6 MB/s 


In [4]:
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Permute, Bidirectional, BatchNormalization, Conv2D, LSTM, Dense, Dropout, ELU, Flatten, MaxPool2D, TimeDistributed, Dense
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

## Import Datasets

In [6]:
X_test = np.load("Dataset/X_test.npy")
y_test = np.load("Dataset/y_test.npy")
X_train_valid = np.load("Dataset/X_train_valid.npy")
y_train_valid = np.load("Dataset/y_train_valid.npy")
person_train_valid = np.load("Dataset/person_train_valid.npy")
person_test = np.load("Dataset/person_test.npy")

## Adjusting the labels 
y_train_valid -= 769
y_test -= 769

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


## Function Definitions

In [7]:
def data_prep(X,y,sub_sample,average,noise):
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,500)
    X = X[:,:,0:500]

    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X, total_y = X_max, y
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    
    # Subsampling
    for i in range(sub_sample):
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
    
    return total_X,total_y


def rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), len(X_train_valid_prep)//8, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test_prep, 4)

    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test
    

def rand_split_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), valid_size, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test_prep


def convert_categorical(x_train, y_train, x_valid, y_valid, X_test_prep, y_test):
    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test, 4)
    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test


def reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test): 
    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test


def sub_selection(idx, X_train, y_train, X_test, y_test, person_train_valid, person_test):
    train_idx = np.where(person_train_valid == idx)[0]
    test_idx = np.where(person_test == idx)[0]
    X_train = X_train[train_idx]
    y_train = y_train[train_idx]
    X_test = X_test[test_idx]
    y_test = y_test[test_idx]
    return X_train, y_train, X_test, y_test


def train(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)

    # Preprocess Data: Training and testing data
    X_train_valid_prep, y_train_valid_prep = data_prep(X_train_valid,y_train_valid,2,2,True)
    X_test_prep, y_test_prep = data_prep(X_test,y_test,2,2,True)

    # Randomly split and reshape: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score

def train_split_first(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)
    
    train_valid_size = len(X_train_valid)
    valid_size = int(train_valid_size * 0.17)

    # Randomly split: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_data(X_train_valid, y_train_valid, X_test, y_test, valid_size)

    # Preprocess: Training, validation, and testing data
    x_train, y_train = data_prep(x_train,y_train,2,2,True)
    x_valid, y_valid = data_prep(x_valid, y_valid,2,2,True)
    x_test, y_test = data_prep(x_test, y_test,2,2,True)

    # Cnvert labels into categorical
    x_train, y_train, x_valid, y_valid, x_test, y_test = convert_categorical(x_train, y_train, x_valid, y_valid, x_test, y_test)

    # Reshape: Training, validation, and testing data
    x_train, y_train, x_valid, y_valid, x_test, y_test = reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score

## CNN model setup

In [8]:
def CNN_model1(kernel_initializer, params):

    lr, dropout_rate = params

    channels_in_1, channels_out_1, kernel_size_1, stride_1 = (250,1,22), 25, (10, 1), 1
    pool_size_1, pool_stride_1 = (3, 1), 1
    channels_out_2, kernel_size_2, stride_2 = 50, (10, 1), 1
    pool_size_2, pool_stride_2 = (3, 1), 1
    channels_out_3, kernel_size_3, stride_3 = 100, (10, 1), 1
    pool_size_3, pool_stride_3 = (3, 1), 1
    channels_out_4, kernel_size_4, stride_4 = 200, (10, 1), 1
    pool_size_4, pool_stride_4 = (3, 1), 1
    hiddendim_1, hiddendim_2, hiddendim_3 = 100, 50, 25

    model = Sequential(
        [
        Conv2D(filters=channels_out_1, kernel_size=kernel_size_1, padding='same', activation='elu', input_shape=channels_in_1),
        MaxPool2D(pool_size=pool_size_1, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_2, kernel_size=kernel_size_2, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_2, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_3, kernel_size=kernel_size_3, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_3, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_4, kernel_size=kernel_size_4, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_4, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),
         
        Flatten(),
        Dense(num_classes, activation='softmax'),
        ]
    )
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(learning_rate=lr),
                  metrics=[categorical_accuracy])
    return model


def CNN_model2(kernel_initializer, params):
    lr = params
    model = Sequential(
        [
        Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu', input_shape=(250,1,22)),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(filters=200, kernel_size=(10,1), padding='same', activation='elu'),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(0.5),
         
        Flatten(),
        Dense(4, activation='softmax'),
        ]
    )
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(learning_rate=lr),
                  metrics=[categorical_accuracy])
    return model

def CNN_model3(kernel_initializer, params):
    lr, dropout_rate, regularizer = params
    model = Sequential(
        [
        Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu', 
               kernel_regularizer=regularizer, input_shape=(250,1,22)),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu', 
               kernel_regularizer=regularizer),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu',
               kernel_regularizer=regularizer),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=200, kernel_size=(10,1), padding='same', activation='elu', 
               kernel_regularizer=regularizer),
        MaxPool2D(pool_size=(3,1), padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),
         
        Flatten(),
         

        Dense(4, activation='softmax'),
        ]
    )
    model.compile(loss='categorical_crossentropy', 
                optimizer=Adam(learning_rate=lr),
                metrics=[categorical_accuracy])
    return model

## CNN Hyper parameter tuning

In [9]:
batch_size = 60
drop_out = 0.2
learning_rate = 1e-3
epoch_num = 30
num_classes = 4

In [10]:
learning_rates = [1e-4, 5e-4, 1e-3]
dropout_rates = [0.2, 0.3, 0.4]
batch_sizes = [60,65]
opt_rs = []  
for i in range(9):
  accuracies = []
  for learning_rate in learning_rates:
    for dropout_rate in dropout_rates:
      for batch_size in batch_sizes:
        cnn_model = CNN_model1(lecun_uniform(seed=42), [learning_rate, drop_out])
        train_score, test_score = train(cnn_model, 
                                      [epoch_num, batch_size],
                                      [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test], idx = i
                                      )
        accuracies.append((i, learning_rate,dropout_rate, batch_size, test_score[1]*100))
  # print(accuracies)
  sort = sorted(accuracies, key = lambda x : x[4])
  opt_rs.append(sort[-1])
print(opt_rs)

Streaming output truncated to the last 5000 lines.
13/13 [==============================] - 0s 12ms/step - loss: 0.0131 - categorical_accuracy: 0.9988 - val_loss: 0.2925 - val_categorical_accuracy: 0.9658
Epoch 30/30
6/6 [==============================] - 0s 5ms/step - loss: 1.2820 - categorical_accuracy: 0.7074
train categorical_accuracy: 100.000%
test categorical_accuracy: 70.745%
Epoch 1/30
14/14 [==============================] - 2s 37ms/step - loss: 1.4930 - categorical_accuracy: 0.4763 - val_loss: 1.0326 - val_categorical_accuracy: 0.5556
Epoch 2/30
14/14 [==============================] - 0s 13ms/step - loss: 0.6046 - categorical_accuracy: 0.7704 - val_loss: 1.2625 - val_categorical_accuracy: 0.5556
Epoch 3/30
14/14 [==============================] - 0s 11ms/step - loss: 0.3119 - categorical_accuracy: 0.8967 - val_loss: 0.4159 - val_categorical_accuracy: 0.8120
Epoch 4/30
14/14 [==============================] - 0s 12ms/step - loss: 0.1893 - categorical_accuracy: 0.9332 - val_lo